# Communicating via TCP

Show how to set up Particle Argon as a TCP Server.

## Particle CLI

### Installation

In [1]:
%%capture
!bash <( curl -sL https://particle.io/install-cli )

### Utility functions

In [2]:
import re
import subprocess

# regular expression to strip ansi control characters
ansi = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')

# decode byte string and strip ansi control characters
def decode_bytes(byte_string):
    if isinstance(byte_string, bytes):
        result = byte_string.decode("utf-8")
    return ansi.sub("", result)

# streamline call to the particle-cli
def particle(args, particle_cli="/root/bin/particle"):
    process = subprocess.run([particle_cli] + args,
                             stdout=subprocess.PIPE,
                             stderr=subprocess.PIPE)
    process.stdout = decode_bytes(process.stdout)
    process.stderr = decode_bytes(process.stderr)
    return process

### Login to Particle

In [3]:
import getpass

# prompt for username and password
username = getpass.getpass(prompt="Username: ")
password = getpass.getpass(prompt="Password: ")

# attempt login
output = particle(["login", "--username", username, "--password", password])

# report results
if output.returncode:
    print(f"Return code = {output.returncode}")
    print(output.stderr)
else:
    print(output.stdout)

Username: ··········
Password: ··········
> Successfully completed login!



### Select a device

The following cell downloads a list of all user devices and creates a list of device names. Here we choose the first name in the list for the rest of this notebook. If this is not the device to be used, then modify this cell accordingly.

In [4]:
devices = [line.split()[0] for line in particle(["list"]).stdout.splitlines()]
device_name = devices[0]
print(particle(["list", device_name]).stdout)

jck_argon_01 [e00fce68eaceb1faa7cf7193] (Argon) is online



## Project Considerations

Using the Particle Device OS, it is straightforward to setup a TCP server which a user can access via standard Windows, MacOS, or Linux utilities. 

## Implementation

MacOS

    nc -4 -n 192.168.xxx.xxx pppp

### Create Project

In [5]:
print(particle(["project", "create", "--name", "myproject", "."]).stdout)

Initializing project in directory myproject...
> A new project has been initialized in directory myproject



### Change working directory

The Particle CLI assumes one is working in the top  project directory.

In [6]:
%cd /content/myproject

/content/myproject


### Create source file

In [81]:
%%writefile src/myproject.ino

/* 
Demonstrate bidirectional communication with Particle using TCPServer and
client. After flashing this firmware, check the Particle console for the local
IP address. Connect to the local IP address using netcat

    nc -4 -n 192.168.xxx.xxx 23

Type H and L will turn the on-board LED on and off.
*/

TCPServer server = TCPServer(23);
TCPClient client;

const int led = D7;
char c;

void setup() {
    server.begin();
    Particle.publish("Server IP", WiFi.localIP().toString().c_str(), PRIVATE);
    pinMode(led, OUTPUT);
}

void loop() {
    if (client.connected()) {
        while (client.available()) {
            /* get next char */
            c = client.read();
            if ('H' == c) {
                server.print("Turn LED on.");
                digitalWrite(led, HIGH);
            } else if ('L' == c) {
                server.print("Turn LED off.");
                digitalWrite(led, LOW);
            } else if ('S' == c) {
                if (digitalRead(led)) {
                    server.print("LED is on.");
                } else {
                    server.print("LED is off.");
                }
            } else if ('X' == c) {
                server.print("Close client connection.");
                client.stop();
            } else {
                server.write(c);
            }
        }
    } else {
        /* get next client */
        client = server.available();
        if (client.connected()) {
            server.println("Enter: ");
            server.println("   H to turn LED on");
            server.println("   L to turn LED off");
            server.println("   S for LED status");
            server.println("   X to close client connection");
        }
    }
}


Overwriting src/myproject.ino


### Compiling

In [82]:
print(particle(["compile", "argon", "--saveTo", "myproject.bin"]).stdout)


Compiling code for argon

Including:
    src/myproject.ino
    project.properties

attempting to compile firmware
downloading binary from: /v1/binaries/5f95ca4704393a21fcca5c6c
saving to: myproject.bin
Memory use:
   text	   data	    bss	    dec	    hex	filename
  11692	    108	   1144	  12944	   3290	/workspace/target/workspace.elf

Compile succeeded.
Saved firmware to: /content/myproject/myproject.bin



### Flash firmware

In [83]:
print(particle(["flash", device_name, "myproject.bin"]).stdout)

Including:
    myproject.bin

attempting to flash firmware to your device jck_argon_01
Flash device OK: Update started

Flash success!

